In [37]:
import torch
from argparse import ArgumentParser
from pprint import pprint
from tqdm import tqdm
import time

from arguments import ModelParams, PipelineParams, OptimizationParams

parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)

parser.add_argument("--my-args")
args = parser.parse_args(args=[
    "--my-args", "my-args",
])

pprint(vars(lp.extract(args)))

print()
print(type(lp), type(lp.extract(args)))
print(args.resolution)
print()

pprint(vars(lp))

{'data_device': 'cuda',
 'eval': False,
 'images': 'images',
 'model_path': '',
 'resolution': -1,
 'sh_degree': 3,
 'source_path': '/home/hjp/KUAICV/NIPS2024_LOCAL/gaussian-splatting',
 'white_background': False}

<class 'arguments.ModelParams'> <class 'arguments.GroupParams'>
-1

{'_images': 'images',
 '_model_path': '',
 '_resolution': -1,
 '_source_path': '',
 '_white_background': False,
 'data_device': 'cuda',
 'eval': False,
 'sh_degree': 3}


In [33]:
model_params, first_iter = torch.load(
    "./output/e0940c85-1/point_cloud/iteration_7000/point_cloud.ply"
)

UnpicklingError: unpickling stack underflow

In [38]:
with tqdm(total=100, desc="Loading model") as pbar:
    for i in range(100):
        time.sleep(0.1)
        pbar.update(1)
        pbar.set_postfix({"i": i})

Loading model: 100%|██████████| 100/100 [00:10<00:00,  9.90it/s, i=99]


In [40]:
print(7_000)

7000
